#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)


In [9]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [10]:
embeddings = OllamaEmbeddings()
db = FAISS.from_documents(docs, embeddings)
db

In [11]:
# querying 
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [12]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [13]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='0665b0d4-2847-4531-aa78-f075738c0648', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(15596.228)),
 (Document(id='3b02c7ab-2ff9-49d5-9a8f-51bf9a610940', metadata={'source': 'spe

In [14]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[1.8891161680221558,
 -3.0950868129730225,
 1.968549370765686,
 1.4706692695617676,
 -0.8890368938446045,
 0.6567972302436829,
 1.5207579135894775,
 -0.4904259443283081,
 0.9170771837234497,
 -2.010498523712158,
 1.3013030290603638,
 -1.9611797332763672,
 -0.6218507885932922,
 1.5266380310058594,
 -0.49796292185783386,
 -2.0463314056396484,
 -0.4233244061470032,
 0.06461568176746368,
 0.7280529141426086,
 -1.8398305177688599,
 -0.6958982348442078,
 -1.03508460521698,
 2.1681134700775146,
 -2.016848564147949,
 0.7546884417533875,
 -0.9740245342254639,
 0.33983784914016724,
 -1.6772277355194092,
 0.16926665604114532,
 -0.8537580370903015,
 2.08321213722229,
 -2.852799654006958,
 -2.6641061305999756,
 3.773005247116089,
 2.2576987743377686,
 -5.105587959289551,
 -1.042070984840393,
 1.3782607316970825,
 -1.2586311101913452,
 -1.3828829526901245,
 -0.4666459858417511,
 -2.042080879211426,
 2.143927574157715,
 0.3474902808666229,
 0.9230367541313171,
 0.5520061254501343,
 -0.257072687149047

In [15]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='0665b0d4-2847-4531-aa78-f075738c0648', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='3b02c7ab-2ff9-49d5-9a8f-51bf9a610940', metadata={'source': 'speech.txt'}, page_content='To 

In [16]:
### Saving And Loading
db.save_local("faiss_index")

In [17]:
new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)

In [18]:
docs

[Document(id='0665b0d4-2847-4531-aa78-f075738c0648', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='3b02c7ab-2ff9-49d5-9a8f-51bf9a610940', metadata={'source': 'speech.txt'}, page_content='To 